Based on https://ymd_h.gitlab.io/ymd_blog/posts/gym_on_google_colab_with_gnwrapper/

## Setup

In [ ]:
!sudo apt-get install -y xvfb ffmpeg x11-utils
!pip install -q 'gym==0.10.11'
!pip install -q 'imageio==2.4.0'
!pip install -q PILLOW
!pip install -q 'pyglet==1.3.2'
!pip install -q pyvirtualdisplay
!pip install -q tf-agents
!pip install colabgymrender
!pip install gym_sokoban
!pip install gym
!apt-get install python-opengl -y
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install piglet

In [ ]:

import gym
import gym_sokoban

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

from colabgymrender.recorder import Recorder
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
Display().start()
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay


import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

In [ ]:
env = gym.make("PushAndPull-Sokoban-v2")
screen = env.render(mode='rgb_array')
print(screen.shape)


In [ ]:
plt.imshow(screen)
# observation = env.reset()
# ipythondisplay.clear_output(wait=True)
# ipythondisplay.display(plt.gcf())

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# pd = Display()
# pd.start()

# directory = './video'
# env = gym.wrappers.Monitor('PushAndPull-Sokoban-v2', "recording",force=True)
# env.render(mode='human')
# env.reset()
# for _ in range(10):
#     action = env.action_space.sample()
#     #print(action)
#     state, reward, done, info = env.step(action) 
#     screen = env.render(mode='rgb_array')
#     plt.imshow(screen)
#     ipythondisplay.clear_output(wait=True)
#     ipythondisplay.display(plt.gcf())
#     if done:
#         env.reset()
#         print('done')


In [ ]:
from skimage import color
# img = color.rgb2gray(screen)
# plt.imshow(img)

In [ ]:
plt.imshow(screen)

In [ ]:
env = gym.make("PushAndPull-Sokoban-v2")
# screen = env.render(mode='rgb_array')
# print(screen.shape)
# plt.imshow(screen)
# observation = env.reset()
# ipythondisplay.clear_output(wait=True)
# ipythondisplay.display(plt.gcf())
# env = gym.make('PushAndPull-Sokoban-v2').unwrapped




In [ ]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [ ]:
class DQN(nn.Module):

    def __init__(self, h, w, outputs):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=5, stride=2)
        self.bn3 = nn.BatchNorm2d(32)

        # Number of Linear input connections depends on output of conv2d layers
        # and therefore the input image size, so compute it.
        def conv2d_size_out(size, kernel_size = 5, stride = 2):
            return (size - (kernel_size - 1) - 1) // stride  + 1
        convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w)))
        convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h)))
        linear_input_size = convw * convh * 32
        self.head = nn.Linear(linear_input_size, outputs)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = x.to(device)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        return self.head(x.view(x.size(0), -1))

In [ ]:
resize = T.Compose([
                    T.ToPILImage(),
                    T.Grayscale(num_output_channels=1),
                    T.Resize(80, interpolation=Image.CUBIC),
                    T.ToTensor()])



def get_screen():
    screen = env.render(mode='rgb_array')#.transpose((2, 0, 1))
    #screen = color.rgb2gray(screen)
    return resize(screen)
    


plt.gray()
plt.imshow(get_screen().cpu().squeeze(0).numpy())
plt.show()


# env.reset()
# plt.figure()
# plt.imshow(get_screen().cpu().squeeze(0).permute(1, 2, 0).numpy(),
#            interpolation='none')
# plt.title('Example extracted screen')
# plt.show()

In [ ]:
plt.figure()
ii = get_screen().squeeze(0)
ii.shape
# plt.imshow(ii.permute(1, 2, 0).numpy(),interpolation='none')

In [ ]:
plt.gray()
plt.imshow(ii.squeeze(0).numpy())
plt.show()
